# Edit Your Expectation Suite
Use this notebook to recreate and modify your expectation suite:

**Expectation Suite Name**: `rating_suite`

We'd love it if you **reach out to us on** the [**Great Expectations Slack Channel**](https://greatexpectations.io/slack)

In [ ]:
import datetime
import great_expectations as ge
import great_expectations.jupyter_ux
from great_expectations.data_context.types.resource_identifiers import (
    ValidationResultIdentifier,
)

context = ge.data_context.DataContext()

# Feel free to change the name of your suite here. Renaming this will not
# remove the other one.
expectation_suite_name = "rating_suite"
suite = context.get_expectation_suite(expectation_suite_name)
suite.expectations = []

batch_kwargs = {
    "data_asset_name": "rating_event",
    "datasource": "rating_test",
    "limit": 1000,
    "schema": "public",
    "table": "rating_event",
}
batch = context.get_batch(batch_kwargs, suite)
batch.head()

## Create & Edit Expectations

Add expectations by calling specific expectation methods on the `batch` object. They all begin with `.expect_` which makes autocompleting easy using tab.

You can see all the available expectations in the **[expectation glossary](https://docs.greatexpectations.io/en/latest/reference/glossary_of_expectations.html?utm_source=notebook&utm_medium=create_expectations)**.

### Table Expectation(s)

In [ ]:
batch.expect_table_row_count_to_be_between(max_value=1100, min_value=900)

In [ ]:
batch.expect_table_column_count_to_equal(value=5)

In [ ]:
batch.expect_table_columns_to_match_ordered_list(
    column_list=["event_id", "kafka_ts", "user_id", "movie_id", "rating"]
)

### Column Expectation(s)

#### `rating`

In [ ]:
batch.expect_column_values_to_not_be_null(column="rating")

In [ ]:
batch.expect_column_distinct_values_to_be_in_set(
    column="rating", value_set=[1, 2, 3, 4, 5]
)

In [ ]:
batch.expect_column_kl_divergence_to_be_less_than(
    column="rating",
    partition_object={
        "values": [1, 2, 3, 4, 5],
        "weights": [0.001, 0.011, 0.193, 0.468, 0.327],
    },
    threshold=0.6,
)

#### `kafka_ts`

In [ ]:
batch.expect_column_values_to_not_be_null(column="kafka_ts")

In [ ]:
batch.expect_column_quantile_values_to_be_between(
    column="kafka_ts",
    allow_relative_error=True,
    quantile_ranges={
        "quantiles": [0.05, 0.25, 0.5, 0.75, 0.95],
        "value_ranges": [
            [1604761953107, 1604761953109],
            [1604762249004, 1604762249006],
            [1604762362185, 1604762362187],
            [1604762439068, 1604762439070],
            [1604762532135, 1604762532137],
        ],
    },
)

#### `movie_id`

In [ ]:
batch.expect_column_values_to_not_be_null(column="movie_id")

In [ ]:
batch.expect_column_value_lengths_to_be_between(column="movie_id", min_value=1)

In [ ]:
batch.expect_column_values_to_match_regex(
    column="movie_id", mostly=0.99, regex="(.*\+\d{4}$)"
)

## Save & Review Your Expectations

Let's save the expectation suite as a JSON file in the `great_expectations/expectations` directory of your project.
If you decide not to save some expectations that you created, use [remove_expectation method](https://docs.greatexpectations.io/en/latest/module_docs/data_asset_module.html?highlight=remove_expectation&utm_source=notebook&utm_medium=edit_expectations#great_expectations.data_asset.data_asset.DataAsset.remove_expectation).

Let's now rebuild your Data Docs, which helps you communicate about your data with both machines and humans.

In [ ]:
batch.save_expectation_suite(discard_failed_expectations=False)

results = context.run_validation_operator("action_list_operator", assets_to_validate=[batch])
validation_result_identifier = results.list_validation_result_identifiers()[0]
context.build_data_docs()
context.open_data_docs(validation_result_identifier)